<a href="https://colab.research.google.com/github/afirth03/IMLOAssessment/blob/main/IMLOExam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Flowers 102 Dataset Classification

In [13]:
import pandas as pd
import numpy as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

train_data = datasets.Flowers102(
    root="data",
    split="train",
    download=True,
    transform=ToTensor()
)

test_data = datasets.Flowers102(
    root="data",
    split="test",
    download=True,
    transform=ToTensor()
)